In [48]:
#coding=utf-8

from collections import defaultdict
from datetime import datetime
import math
from operator import itemgetter
import os
import random
import re

import numpy as np
import pandas as pd

%matplotlib inline

import little_mallet_wrapper as lmw
import openpyxl

import copy
import csv

import xlwt
import xlrd
import xlutils.copy as xc

import os

# 1 MALLET Path

In [49]:
path_to_mallet = 'C:/mallet/bin/mallet'  # CHANGE THIS TO YOUR MALLET PATH

# 2 Load dataset

16张sheet，每张1大类


In [50]:
dataset_path = '../huawei_appdescrip_spider/4-大类LDA-应用-华为应用市场app分类与功能描述.xls'  # CHANGE THIS TO YOUR DATASET PATH

df_sheets_dict = pd.read_excel(dataset_path,sheet_name = None,encoding="utf-8")
sheets_name = list(df_sheets_dict.keys())

In [51]:
df_sheets = [df_sheets_dict[i] for i in sheets_name] #16张表

# 从表格读取数据到list，待输入LDA
training_data_list = []
for df_sheet in df_sheets:
    training_data = [t for t in df_sheet['迭代去干扰词4'].tolist()]
    # training_data = [t for t in df_sheet['分词+去停用词+去低频词'].tolist()]
    training_data = [str(d).strip() for d in training_data if str(d)]
    training_data_list.append(training_data)

# training_data_list    

In [52]:
num_sub_types = [6,6,4,4,7,4,4,5,3,6,5,3,5,4,6,4] #每大类分成几小类

# 3 Train topic model

## 3.1 训练 LDA 主题模型

In [53]:
for i in range(16):    
    num_topics = num_sub_types[i]  # CHANGE THIS TO YOUR PREFERRED NUMBER OF TOPICS
    path_str = 'huawei_LDA_output_原分类测试/'+ sheets_name[i] + '-' + str(num_topics)
    os.makedirs(path_str)

    output_directory_path = path_str

    path_to_training_data           = output_directory_path + '/training.txt'
    path_to_formatted_training_data = output_directory_path + '/mallet.training'
    path_to_model                   = output_directory_path + '/mallet.model.' + str(num_topics)
    path_to_topic_keys              = output_directory_path + '/mallet.topic_keys.' + str(num_topics)
    path_to_topic_distributions     = output_directory_path + '/mallet.topic_distributions.' + str(num_topics)
    path_to_word_weights            = output_directory_path + '/mallet.word_weights.' + str(num_topics)
    path_to_diagnostics             = output_directory_path + '/mallet.diagnostics.' + str(num_topics) + '.xml'

    
    lmw.import_data(path_to_mallet,
                    path_to_training_data, #txt格式数据
                    path_to_formatted_training_data, #MALLET 格式数据
                    training_data_list[i]) #原始数据

    print("开始训练",i+1,sheets_name[i],"LDA模型...")
    lmw.train_topic_model(path_to_mallet,
                      path_to_formatted_training_data,
                      path_to_model,
                      path_to_topic_keys,
                      path_to_topic_distributions,
                      path_to_word_weights,
                      path_to_diagnostics,
                      num_topics)



Importing data...
Complete
开始训练 1 影音娱乐 LDA模型...
Training topic model...
Complete
Importing data...
Complete
开始训练 2 实用工具 LDA模型...
Training topic model...
Complete
Importing data...
Complete
开始训练 3 社交通讯 LDA模型...
Training topic model...
Complete
Importing data...
Complete
开始训练 4 教育 LDA模型...
Training topic model...
Complete
Importing data...
Complete
开始训练 5 新闻阅读 LDA模型...
Training topic model...
Complete
Importing data...
Complete
开始训练 6 拍摄美化 LDA模型...
Training topic model...
Complete
Importing data...
Complete
开始训练 7 美食 LDA模型...
Training topic model...
Complete
Importing data...
Complete
开始训练 8 出行导航 LDA模型...
Training topic model...
Complete
Importing data...
Complete
开始训练 9 旅游住宿 LDA模型...
Training topic model...
Complete
Importing data...
Complete
开始训练 10 购物比价 LDA模型...
Training topic model...
Complete
Importing data...
Complete
开始训练 11 商务 LDA模型...
Training topic model...
Complete
Importing data...
Complete
开始训练 12 儿童 LDA模型...
Training topic model...
Complete
Importing data...
Complete
开始训练 1

## 3.2 doc主题分布

### 1) 加载每个doc的主题分布

In [54]:
topic_distributions_list = []

for i in range(16):
    num_topics = num_sub_types[i]  
    output_directory_path = 'huawei_LDA_output_原分类测试/'+ sheets_name[i] + '-' + str(num_topics) 

    topic_distributions = lmw.load_topic_distributions(output_directory_path + '/mallet.topic_distributions.' + str(num_topics))
    topic_distributions_list.append(topic_distributions)

### 2)获取每个doc的的主题向量：概率从大到小排列

In [55]:
indexs_max_p_all_list = []

for k in range(16):
    num_topics = num_sub_types[k]  
    output_directory_path = 'huawei_LDA_output_原分类测试/'+ sheets_name[k] + '-' + str(num_topics) 
    # print(sheets_name[k])

    num_max_p_topics =  num_sub_types[k]#取每个doc最可能的topic个数

    indexs_max_p_all = [] #存所有doc的主题向量，主题向量=[(主题,概率),()]

    for i in range(len(training_data_list[k])): #遍历所有doc
        p_topics = copy.deepcopy(topic_distributions_list[k][i]) #每个doc的概率list
        indexs_max_p = [] #存1个doc最可能的topic下标

        for j in range(num_max_p_topics): #找最大值
            max_p = max(p_topics)
            index_max_p = p_topics.index(max_p)

            # indexs_max_p.append((index_max_p,max_p)) #(主题下标，概率) 
            indexs_max_p.append((index_max_p,round(max_p,4))) #(主题下标，概率)
            # indexs_max_p.append((index_max_p,round(max_p,4)*100)) #(主题下标，概率) 概率单位%
            p_topics[index_max_p] = -1 #最大值改0

        indexs_max_p_all.append(indexs_max_p)
    
    indexs_max_p_all_list.append(indexs_max_p_all)

### 3)追加写入主题向量到excel

In [56]:
def write_excel_xls_append(path, id_sheet,value):
    index = len(value)  # 获取需要写入数据的行数

    workbook = xlrd.open_workbook(path)  # 打开工作簿
    sheets = workbook.sheet_names()  # 获取工作簿中的所有表格
    worksheet = workbook.sheet_by_name(sheets[id_sheet])  # 获取工作簿中所有表格中的的第一个表格

    rows_old = worksheet.nrows  # 获取表格中已存在的数据的行数
    cols_old = worksheet.ncols  # 获取表格中已存在的数据的行数

    new_workbook = xc.copy(workbook)  # 将xlrd对象拷贝转化为xlwt对象
    new_worksheet = new_workbook.get_sheet(id_sheet)  # 获取转化后工作簿中的第一个表格

    for i in range(0, index):
        for j in range(0, len(value[i])):
            new_worksheet.write(i, j+cols_old, str(value[i][j]))  # 追加写入数据，注意是从i+rows_old行开始写入
    new_workbook.save(path)  # 保存工作簿
    print("写入完成！")

In [58]:
for k in range(16):
    num_topics = num_sub_types[k]
    print("写入主题向量:",sheets_name[k],"...")

    data = []
    data.append(["主题","概率"])

    temp_indexs_max_p_all = indexs_max_p_all_list[k]
    maxnum = len(temp_indexs_max_p_all)

    for i in range(maxnum): 
        topic_vector = copy.deepcopy(temp_indexs_max_p_all[i]) #遍历每个app的主题向量=[(主题,概率),()]
        temp = []

        for j in range(num_topics): #遍历主题向量内每个元组，共num_topics个
            if topic_vector[j][1] <= 0.05: 
                topic_vector = topic_vector[:j]
                break
            else:
                temp.append(topic_vector[j][0])
                temp.append(topic_vector[j][1])

        data.append(temp)
        
    write_excel_xls_append("LDA-应用-主题向量-原分类测试.xls",k,data)



写入主题向量: 影音娱乐 ...
写入完成！
写入主题向量: 实用工具 ...
写入完成！
写入主题向量: 社交通讯 ...
写入完成！
写入主题向量: 教育 ...
写入完成！
写入主题向量: 新闻阅读 ...
写入完成！
写入主题向量: 拍摄美化 ...
写入完成！
写入主题向量: 美食 ...
写入完成！
写入主题向量: 出行导航 ...
写入完成！
写入主题向量: 旅游住宿 ...
写入完成！
写入主题向量: 购物比价 ...
写入完成！
写入主题向量: 商务 ...
写入完成！
写入主题向量: 儿童 ...
写入完成！
写入主题向量: 金融理财 ...
写入完成！
写入主题向量: 运动健康 ...
写入完成！
写入主题向量: 便捷生活 ...
写入完成！
写入主题向量: 汽车 ...
写入完成！


### 4)属于指定topic的概率最高的doc

In [ ]:
# get_top_docs返回元组(概率，训练的doc)
for p, d in lmw.get_top_docs(training_data, topic_distributions, topic_index=0, n=3):
    print(round(p, 4), d) #概率取小数点后4位
    print()

## 3.3 主题词

- 为每个主题加载概率最大单词集：每个主题最多20个词


In [59]:
topic_keys_list = []
for i in range(16):
    num_topics = num_sub_types[i]  
    output_directory_path = 'huawei_LDA_output_原分类测试/'+ sheets_name[i] + '-' + str(num_topics) 
    topic_keys_path = output_directory_path + '/mallet.topic_keys.' + str(num_topics)
    topic_keys = [line.split('\t')[2].split() for line in open(topic_keys_path, 'r',encoding="utf-8")] 

    topic_keys_list.append(topic_keys)

- 加载每个topic的最高概率主题词分布

In [60]:
topic_keys_list = []

for i in range(16):
    num_topics = num_sub_types[i]  
    output_directory_path = 'huawei_LDA_output_原分类测试/'+ sheets_name[i] + '-' + str(num_topics) 
    word_weight_path = output_directory_path + '/mallet.word_weights.' + str(num_topics)

    topic_word_weight_dict = defaultdict(lambda: defaultdict(float))
    topic_sum_dict = defaultdict(float)
    with open(word_weight_path,'r',encoding="utf-8") as f:       
        for _line in f:        
            _topic, _word, _weight = _line.split('\t')
            topic_word_weight_dict[_topic][_word] = float(_weight)
            topic_sum_dict[_topic] += float(_weight)

    topic_word_probability_dict = defaultdict(lambda: defaultdict(float))
    for _topic, _word_weight_dict in topic_word_weight_dict.items():
        for _word, _weight in _word_weight_dict.items():
            topic_word_probability_dict[int(_topic)][_word] = _weight / topic_sum_dict[_topic]

    print("=====================",sheets_name[i],"共",len(topic_word_probability_dict),"个topic==================")

    for _topic, _word_probability_dict in topic_word_probability_dict.items():
        print('Topic', _topic)
        for _word, _probability in sorted(_word_probability_dict.items(), key=lambda x: x[1], reverse=True)[:5]:
            print(round(_probability, 4), '\t', _word)
        print()

===================== 影音娱乐 共 6 个topic==================
Topic 0
0.0382 	 娃娃机
0.0272 	 音乐人
0.0257 	 互联网
0.0214 	 电视台
0.017 	 马戏团

Topic 1
0.053 	 收音机
0.0205 	 广场舞
0.0156 	 脱口秀
0.0139 	 爱好者
0.0123 	 广播剧

Topic 2
0.0868 	 高颜值
0.0167 	 ktv
0.0156 	 面对面
0.0134 	 演唱会
0.0134 	 陌生人

Topic 3
0.0374 	 电视剧
0.0155 	 纪录片
0.0117 	 电视台
0.0117 	 真人秀
0.0104 	 年轻人

Topic 4
0.0895 	 节拍器
0.0639 	 调音器
0.0474 	 尤克里里
0.0346 	 爱好者
0.0174 	 吉他谱

Topic 5
0.1158 	 播放器
0.0302 	 均衡器
0.0212 	 音频格式
0.0151 	 wifi
0.0127 	 音乐库

===================== 实用工具 共 6 个topic==================
Topic 0
0.0566 	 输入法
0.0356 	 摄像头
0.0331 	 朋友圈
0.0315 	 倒计时
0.0191 	 联系人

Topic 1
0.0735 	 二维码
0.0507 	 手电筒
0.0254 	 闪光灯
0.0154 	 zip
0.0138 	 播放器

Topic 2
0.2469 	 wifi
0.0336 	 加速器
0.0285 	 路由器
0.0279 	 语音包
0.0246 	 公积金

Topic 3
0.0865 	 遥控器
0.0209 	 nfc
0.0122 	 投影仪
0.0118 	 互联网
0.0114 	 机顶盒

Topic 4
0.0764 	 浏览器
0.0348 	 备忘录
0.0266 	 指南针
0.0125 	 录音转
0.0113 	 水平仪

Topic 5
0.1406 	 计算器
0.0823 	 pdf
0.0386 	 打印机
0.0351 	 word
0.0163 	 pp

## 3.4 主题分布散度

- 计算两个topic分布之间的 Jensen-Shannon 散度

In [ ]:
lmw.get_js_divergence_topics(0, 7, topic_word_probability_dict)

# Plot topics by category

如果数据集包含一段时间的分类标签，则创建标签 x 主题的热图

In [ ]:
target_labels = ['John Keats', 'Emily Dickinson', 'William Butler Yeats', 'Christina Rossetti']

lmw.plot_categories_by_topics_heatmap(authors,
                                      topic_distributions,
                                      topic_keys, 
                                      output_directory_path + '/categories_by_topics.pdf',
                                      target_labels=target_labels,
                                      dim=(12,4))

如果数据集包含一些时间的分类标签，则创建一组箱线图，每个主题一个图。

In [ ]:
target_labels = ['John Keats', 'Emily Dickinson', 'William Butler Yeats', 'Christina Rossetti']

for _topic_index in range(0, len(topic_keys)):
    
    lmw.plot_categories_by_topic_boxplots(authors,
                                          topic_distributions,
                                          topic_keys, 
                                          _topic_index,
                                          output_path=output_directory_path + '/boxplot.' + str(_topic_index) + '.pdf',
                                          target_labels=target_labels,
                                          dim=(4,4))

# Plot topics over document time

Splits each training document into N chunks, infers the topics for those chunks (using the pre-trained topic model), and plots mean topic probabilities over document time.

In the example shown, it looks like poems often start with physical descriptions and end with abstract and romantic language.

给定一个数据集，将每个文档分成num_chunks个doc

In [ ]:
divided_documents, document_ids, times = lmw.divide_training_data(training_data,
                                                                  num_chunks=10)

len(divided_documents), len(document_ids), len(times)

In [ ]:
path_to_divided_training_data           = output_directory_path + '/training.split.txt'
path_to_divided_formatted_training_data = output_directory_path + '/mallet.split.training'
path_to_divided_topic_distributions     = output_directory_path + '/mallet.split.topic_distributions.' + str(num_topics)

In [ ]:
lmw.import_data(path_to_mallet,
                path_to_divided_training_data,
                path_to_divided_formatted_training_data,
                divided_documents,
                use_pipe_from=path_to_formatted_training_data)

# **模型使用**
使用已在另一组文档上训练的模型获取一组新文档的主题分布

In [ ]:
lmw.infer_topics(path_to_mallet,
                 path_to_model,
                 path_to_divided_formatted_training_data,
                 path_to_divided_topic_distributions)

在训练主题模型后加载每个文档的主题分布

In [ ]:
topic_distributions = lmw.load_topic_distributions(path_to_divided_topic_distributions)

len(topic_distributions), len(topic_distributions[2])

创建线图，每个主题一个，显示文档段的平均主题概率

In [ ]:
for _topic_index in range(0, len(topic_keys)):
    lmw.plot_topics_over_time(topic_distributions, topic_keys, times, _topic_index)